In [1]:
import requests
import pandas as pd
import numpy as np
import leafmap
import re
import geopandas as gpd

import folium
from folium import Marker
from folium.plugins import FastMarkerCluster
from folium.plugins import MarkerCluster
pd.set_option('display.max_columns', None)

In [2]:
#https://www.data.gouv.fr/fr/datasets/bornes-de-chargement-publiques-pour-voitures-electriques-du-plusieurs-operateurs/
url = "https://www.data.gouv.fr/fr/datasets/r/b11113db-875d-41c7-8673-0cf8ad43e917"
nom_fichier = "Src_bornes_gouv.csv"

response = requests.get(url)

if response.status_code == 200:
    with open(nom_fichier, 'wb') as fichier:
        fichier.write(response.content)
    print("Téléchargement réussi.")
else:
    print("Échec du téléchargement. Code de statut :", response.status_code)

Téléchargement réussi.


In [3]:

    
url2 = "https://france-geojson.gregoiredavid.fr/repo/departements.geojson"
nom_fichier = "Src_depart_gouv.geojson"

response = requests.get(url2)

if response.status_code == 200:
    with open(nom_fichier, 'wb') as fichier:
        fichier.write(response.content)
    print("Téléchargement réussi.")
else:
    print("Échec du téléchargement. Code de statut :", response.status_code)

Téléchargement réussi.


In [4]:
df = pd.read_csv('Src_bornes_gouv.csv')

df.sample(10)

,contact_amenageur,contact_operateur,nom_amenageur,nom_operateur,telephone_operateur,siren_amenageur,raccordement,nom_enseigne,id_station_itinerance,id_station_local,nom_station,implantation_station,adresse_station,coordonneesXY,nbre_pdc,id_pdc_itinerance,id_pdc_local,puissance_nominale,prise_type_ef,prise_type_2,prise_type_combo_ccs,prise_type_chademo,prise_type_autre,gratuit,tarification,paiement_autre,paiement_acte,paiement_cb,condition_acces,reservation,horaires,accessibilite_pmr,restriction_gabarit,station_deux_roues,date_mise_en_service,observations,date_maj,cable_t2_attache
7898,support@modulo-energies.fr,support@modulo-energies.fr,Modulo Energies,Modulo,0800 37 37 40,841376734.0,NaN,55 BAR LE DUC-parking de la gare-66183,FRS55E6619,FRS55E6619,55 BAR LE DUC-parking de la gare-66183,Voirie,"Parking de la Gare, 55000 BAR LE DUC","[48.77318600,5.16697300]",4,FRS55E66191,FRS55E66191,18,True,True,False,False,False,False,NaN,True,True,False,Accès réservé,False,24/7,Accessibilité inconnue,inconnu,False,2021-09-07,NaN,2023-09-30,False
5697,kontakt@lidl.de,kontakt@lidl.de,NaN,NaN,+(33)-(800)-900343,NaN,NaN,Lidl SAINT JULIEN EN GENEVOIS,FRLDLE0000376,FRLDLE0000376,Lidl SAINT JULIEN EN GENEVOIS,Station dédiée à la recharge rapide,"Route de Lathoy 6, 74160 Saint-Julien-en-Genevois","[46.14140000,6.09489800]",6,FRLDLE00003762,FRLDLE00003762,60,False,False,True,False,False,False,NaN,NaN,True,False,Accès libre,True,"Mo 00:00-23:59,Tu 00:00-23:59,We 00:00-23:59,T...",Accessibilité inconnue,inconnu,False,2022-08-16,NaN,2023-10-01,False
7146,info@ionity.eu,info@ionity.eu,NaN,NaN,+(33)-(1)-87210891,NaN,NaN,Pamiers,FRIOYE4466,FRIOYE4466,Pamiers,Station dédiée à la recharge rapide,"Parc Commercial Delta-Sud ZA de la Grausette, ...","[43.07488300,1.64161700]",7,FRIOYE44665,FRIOYE44665,50,False,False,True,True,False,False,NaN,NaN,True,True,Accès libre,False,24/7,Accessibilité inconnue,inconnu,False,2023-03-09,NaN,2023-09-30,False
8073,info@allego.eu,info@allego.eu,NaN,NaN,+(33)-(800)-78192,NaN,NaN,Allego Carrefour Feurs,FRALLEGO900599,FRALLEGO900599,Allego Carrefour Feurs,Voirie,"Route de Saint Etienne_RN 82_BP 76, 42110 Feurs","[45.72129000,4.22727000]",8,FRALLEGO9005991,FRALLEGO9005991,22,False,True,False,False,False,False,NaN,NaN,True,True,Accès libre,False,24/7,Accessibilité inconnue,inconnu,False,2022-12-08,NaN,2023-09-30,False
4730,support@modulo-energies.fr,support@modulo-energies.fr,Modulo Energies,Modulo,0800 37 37 40,841376734.0,NaN,COMMERCY - 158012 - Piscine,FRS55E15800,FRS55E15800,COMMERCY - 158012 - Piscine,Voirie,"Rue du Colonel Beltrame, 55200 Commercy","[48.75507600,5.60222700]",12,FRS55E158007,FRS55E158007,18,True,True,False,False,False,False,NaN,True,True,False,Accès libre,False,24/7,Accessibilité inconnue,inconnu,False,2022-10-10,NaN,2023-10-01,False
8524,info@allego.eu,info@allego.eu,NaN,NaN,+(33)-(800)-78192,NaN,NaN,Allego Carrefour Claye-Souilly,FRALLEGO900706,FRALLEGO900706,Allego Carrefour Claye-Souilly,Voirie,"Route Nationale 3_BP 70, 77413 Claye-Souilly","[48.94737000,2.66373000]",8,FRALLEGO9007061,FRALLEGO9007061,22,False,True,False,False,False,False,NaN,NaN,True,True,Accès libre,False,24/7,Accessibilité inconnue,inconnu,False,2022-10-27,NaN,2023-09-30,False
6408,info@ionity.eu,info@ionity.eu,NaN,NaN,+(33)-(1)-87210891,NaN,NaN,Chaberte La Garde,FRIOYE45180,FRIOYE45180,Chaberte La Garde,Station dédiée à la recharge rapide,"Autoroute A57, aire de La garde, 83130 La Garde","[43.14877000,6.03343700]",17,FRIOYE451805,FRIOYE451805,350,False,False,True,False,False,False,NaN,NaN,True,True,Accès libre,False,24/7,Accessibilité inconnue,inconnu,False,2023-07-05,NaN,2023-09-30,False
9007,kontakt@lidl.de,kontakt@lidl.de,NaN,NaN,+(33)-(800)-900343,NaN,NaN,Lidl ST ELOY LES MINES Manoir,FRLDLEFRLDLE0000292,FRLDLEFRLDLE0000292,Lidl ST ELOY LES MINES Manoir,Voirie,"4 Rue du Puits du Manoir, 63700 Saint-Éloy-les...","[46.16541100,2.83275300]",6,FRLDLEFRLDLE00002927,FRLDLEFRLDLE00002927,22,False,True,False,False,False,False,NaN,NaN,Tr

In [5]:
df[['x', 'y']] = df['coordonneesXY'].str.strip('[]').str.split(',', expand=True).astype(float)

def extraire_cp(texte):
    match = re.search(r'\d{5}', texte)
    if match:
        chiffres = match.group()
        return chiffres[:2]  # Prendre les deux premiers chiffres
    else:
        return None

# Appliquez la fonction à la colonne 'Adresse' du DataFrame
df['Departement'] = df['adresse_station'].apply(extraire_cp)



In [6]:
df_region = pd.read_csv('POP_donnees_communes.csv',sep=";")
df_region=df_region[['REG', 'CODDEP',  'COM', 'PTOT']]
df_region = df_region.groupby('CODDEP').sum().reset_index()
df_region

,CODDEP,PTOT
0,01,668565
1,02,543368
2,03,344922
3,04,168867
4,05,145791
...,...,...
95,95,1259985
96,971,389995
97,972,369406
98,973,284085


In [7]:
df_count_stat = df.groupby('Departement')['adresse_station'].count()
df_region = pd.merge(df_region,df_count_stat,left_on='CODDEP',right_on='Departement')
df_region['Nb stat pour 100khab'] = round((df_region['adresse_station']*100000)/df_region['PTOT'])
df_region

,CODDEP,PTOT,adresse_station,Nb stat pour 100khab
0,01,668565,101,15.0
1,02,543368,70,13.0
2,03,344922,55,16.0
3,04,168867,28,17.0
4,05,145791,15,10.0
...,...,...,...,...
89,91,1316011,147,11.0
90,92,1640721,41,2.0
91,93,1651739,47,3.0
92,94,1415845,72,5.0


In [8]:
df_region.columns


Index(['CODDEP', 'PTOT', 'adresse_station', 'Nb stat pour 100khab'], dtype='object')

In [9]:
df_map=df[['nom_station', 'implantation_station',
       'adresse_station', 'nbre_pdc', 
       'puissance_nominale', 'prise_type_ef', 'prise_type_2',
       'prise_type_combo_ccs', 'prise_type_chademo', 'prise_type_autre',
       'gratuit', 'tarification', 'paiement_autre', 'paiement_acte',
       'paiement_cb', 'condition_acces', 'reservation', 'horaires',
       'accessibilite_pmr', 'restriction_gabarit', 'station_deux_roues',
       'date_mise_en_service', 'observations', 'date_maj', 'cable_t2_attache','x','y']]

In [ ]:



ma_carte = leafmap.Map(center=[45.76077534099906, 4.833680068867414],zoom=12)

ma_carte.add_geojson(url2, layer_name='Departements')
ma_carte.add_points_from_xy(data=df_map, x='y', y='x', radius=10)
ma_carte.add_data(data=df_region, colors='Nb stat pour 100khab')


In [11]:
ma_carte.save('output.html')

In [12]:
df.columns[[
       'raccordement', 'nom_enseigne', 
        'nom_station', 'implantation_station',
       'adresse_station', 'nbre_pdc', 
       'puissance_nominale', 'prise_type_ef', 'prise_type_2',
       'prise_type_combo_ccs', 'prise_type_chademo', 'prise_type_autre',
       'gratuit', 'tarification', 'paiement_autre', 'paiement_acte',
       'paiement_cb', 'condition_acces', 'reservation', 'horaires',
       'accessibilite_pmr', 'restriction_gabarit', 'station_deux_roues',
       'date_mise_en_service', 'observations', 'date_maj', 'cable_t2_attache',
       'x', 'y']]


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
df_map=df[['nom_station', 'implantation_station',
       'adresse_station', 'nbre_pdc', 
       'puissance_nominale', 'prise_type_ef', 'prise_type_2',
       'prise_type_combo_ccs', 'prise_type_chademo', 'prise_type_autre',
       'gratuit', 'tarification', 'paiement_autre', 'paiement_acte',
       'paiement_cb', 'condition_acces', 'reservation', 'horaires',
       'accessibilite_pmr', 'restriction_gabarit', 'station_deux_roues',
       'date_mise_en_service', 'observations', 'date_maj', 'cable_t2_attache','x','y']]

In [ ]:
df['raccordement'].unique

In [14]:
df_region['Nb stat pour 100khab']


0     15.0
1     13.0
2     16.0
3     17.0
4     10.0
      ... 
89    11.0
90     2.0
91     3.0
92     5.0
93    15.0
Name: Nb stat pour 100khab, Length: 94, dtype: float64